In [14]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os 
from warnings import simplefilter
import pandas as pd

In [15]:
import models
import class_sampling
import train
import metric_utils
import inference

In [16]:
NUM_CLASSES = 10
n_epochs = 50
batch_size_train = 64
batch_size_test = 1000
momentum = 0

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

NUM_CLASSES_REDUCED = 2
nums = (0, 1)
ratio = (100, 1)
#nums = (0, 3, 1)
#ratio = (200, 20, 1)

CLASS_LABELS = {'airplane': 0,
                 'automobile': 1,
                 'bird': 2,
                 'cat': 3,
                 'deer': 4,
                 'dog': 5,
                 'frog': 6,
                 'horse': 7,
                 'ship': 8,
                 'truck': 9}


simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=DeprecationWarning)

In [17]:
# name, num_classes, classes used, ratio, learning rate, mean 10, variance 10, mean 20, variance 20, ... 50

# mean, variance every 10 epochs - average of 10 models 
# name, num_classes, classes used, ratio, learning rate, mean 10, variance 10, mean 20, variance 20, ... 50
# name: normal/ratio/oversampled/undersampled/weighted

col_names = ["name", 
            "num_classes", 
            "classes_used", 
            "ratio", 
            "learning_rate", 
            "mean_0", "variance_0",
            "mean_10", "variance_10",
            "mean_20", "variance_20",
            "mean_30", "variance_30",
            "mean_40", "variance_40",
            "mean_50", "variance_50"]

rows = []

In [18]:
train_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor() ]))  


test_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()]))

train_CIFAR10.data = train_CIFAR10.data.reshape(50000, 3, 32, 32)
test_CIFAR10.data = test_CIFAR10.data.reshape(10000, 3, 32, 32)

    
reduced_train_CIFAR10 = class_sampling.Reduce(train_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True)
reduced_test_CIFAR10 = class_sampling.Reduce(test_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True)

ratio_train_CIFAR10 = class_sampling.Ratio(train_CIFAR10, NUM_CLASSES_REDUCED, ratio, nums=nums)

Files already downloaded and verified
Files already downloaded and verified


In [19]:
targets = ratio_train_CIFAR10.labels 

class_count = np.unique(targets, return_counts=True)[1]
print(class_count)

weight = 1. / class_count

samples_weight = weight[targets]
samples_weight = torch.from_numpy(samples_weight)
oversampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(max(class_count) * NUM_CLASSES_REDUCED), replacement=True)
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)
undersampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * NUM_CLASSES_REDUCED), replacement=False)

weight *= class_count[0]
print(weight)

[5000   50]
[  1. 100.]


In [20]:
train_loader_reduced = DataLoader(reduced_train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_ratio = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True) 

train_loader_oversampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=oversampler)

train_loader_undersampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler)

train_loader_sampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=sampler)

test_loader_reduced = DataLoader(reduced_test_CIFAR10, batch_size=batch_size_test, shuffle=True)

In [21]:
print(train_loader_oversampled.dataset.labels.shape)

torch.Size([5050])


In [83]:
# SIGMOID 2 CLASS normal AUC saving  
# df.to_csv()

momentum=0
learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SigmoidLogisticRegression(NUM_CLASSES_REDUCED, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_reduced, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)
   


Test set: Avg. loss: 0.02157968044281006, AUC: 0.6867034999999999


Test set: Avg. loss: 0.03557538032531738, AUC: 0.570759


Test set: Avg. loss: 0.013752534389495849, AUC: 0.889133


Test set: Avg. loss: 0.007762441396713257, AUC: 0.8451270000000001


Test set: Avg. loss: 0.008736375808715821, AUC: 0.8208995


Test set: Avg. loss: 0.008814629077911376, AUC: 0.8945854999999999


Test set: Avg. loss: 0.023658716201782226, AUC: 0.398378


Test set: Avg. loss: 0.12261966705322265, AUC: 0.532487


Test set: Avg. loss: 0.00910294771194458, AUC: 0.8903105


Test set: Avg. loss: 0.00748134970664978, AUC: 0.8410195


Test set: Avg. loss: 0.048084203720092776, AUC: 0.7297494999999999


Test set: Avg. loss: 0.007408306837081909, AUC: 0.848622


Test set: Avg. loss: 0.04466411209106445, AUC: 0.5848144999999999


Test set: Avg. loss: 0.009844433307647706, AUC: 0.8901745


Test set: Avg. loss: 0.011238266944885254, AUC: 0.7808945


Test set: Avg. loss: 0.010974647521972657, AUC: 0.896553500000000

KeyboardInterrupt: 

In [ ]:
print(learning_rate_aucs.shape)
auc_mean = np.mean(learning_rate_aucs, axis=1)
print(auc_mean.shape)
auc_variance = np.var(learning_rate_aucs, axis=1)
print(auc_variance.shape)

for i in range(len(learning_rates)): 
    row = ["normal", 2, nums, (1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)

print(rows)

# pd.DataFrame(rows, columns = col_names) 

In [ ]:
# SIGMOID 2 CLASS ratio

momentum=0
learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SigmoidLogisticRegression(2, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["ratio", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)


In [91]:
# SIGMOID 2 CLASS oversampled

momentum=0
learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SigmoidLogisticRegression(2, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_oversampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["oversampled", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)



Test set: Avg. loss: 0.02157968044281006, AUC: 0.6867034999999999


Test set: Avg. loss: 0.020572117805480957, AUC: 0.8238075


Test set: Avg. loss: 0.02619137954711914, AUC: 0.8130915


Test set: Avg. loss: 0.04026922607421875, AUC: 0.752547


Test set: Avg. loss: 0.027536532402038575, AUC: 0.7995220000000001


Test set: Avg. loss: 0.029943819046020507, AUC: 0.790417


Test set: Avg. loss: 0.07544555282592773, AUC: 0.497509


Test set: Avg. loss: 0.07978406143188477, AUC: 0.5938809999999999


Test set: Avg. loss: 0.024683547973632813, AUC: 0.8177650000000001


Test set: Avg. loss: 0.025677730560302734, AUC: 0.8069000000000001


Test set: Avg. loss: 0.019481823921203612, AUC: 0.8099750000000001


Test set: Avg. loss: 0.15527237701416016, AUC: 0.502493


Test set: Avg. loss: 0.06202663040161133, AUC: 0.4889945


Test set: Avg. loss: 0.01783221435546875, AUC: 0.8321440000000001


Test set: Avg. loss: 0.024161662101745605, AUC: 0.815463


Test set: Avg. loss: 0.026190677642822267, AUC: 0


Test set: Avg. loss: 0.007803581714630127, AUC: 0.7509425


Test set: Avg. loss: 0.02522551441192627, AUC: 0.687897


Test set: Avg. loss: 0.005271515130996704, AUC: 0.7197325


Test set: Avg. loss: 0.005499098300933838, AUC: 0.726059


Test set: Avg. loss: 0.006402374267578125, AUC: 0.728049


Test set: Avg. loss: 0.006547322034835815, AUC: 0.7265980000000001


Test set: Avg. loss: 0.007059020757675171, AUC: 0.7297285


Test set: Avg. loss: 0.009229622840881347, AUC: 0.604512


Test set: Avg. loss: 0.005724687099456787, AUC: 0.690681


Test set: Avg. loss: 0.00515353512763977, AUC: 0.6875884999999999


Test set: Avg. loss: 0.006271977186203003, AUC: 0.7067395


Test set: Avg. loss: 0.00663629937171936, AUC: 0.7148905000000001


Test set: Avg. loss: 0.007044186592102051, AUC: 0.7193314999999999


Test set: Avg. loss: 0.05417813491821289, AUC: 0.4504045


Test set: Avg. loss: 0.0052720818519592285, AUC: 0.7127730000000001


Test set: Avg. loss: 0.0050723919868469235, AUC: 0.71127049999


Test set: Avg. loss: 0.0067416255474090575, AUC: 0.6962735


Test set: Avg. loss: 0.006592718601226807, AUC: 0.7002335


Test set: Avg. loss: 0.012907815456390382, AUC: 0.575434


Test set: Avg. loss: 0.011563073635101319, AUC: 0.591401


Test set: Avg. loss: 0.0106242995262146, AUC: 0.6025739999999999


Test set: Avg. loss: 0.009890056610107422, AUC: 0.6133085


Test set: Avg. loss: 0.009341772079467774, AUC: 0.623823


Test set: Avg. loss: 0.00890287971496582, AUC: 0.6340315000000001


Test set: Avg. loss: 0.026033576011657716, AUC: 0.422333


Test set: Avg. loss: 0.013315052986145019, AUC: 0.5064865000000001


Test set: Avg. loss: 0.01190878963470459, AUC: 0.5262104999999999


Test set: Avg. loss: 0.01089624547958374, AUC: 0.5509645000000001


Test set: Avg. loss: 0.00981254768371582, AUC: 0.568669


Test set: Avg. loss: 0.009073453903198242, AUC: 0.5884425


Test set: Avg. loss: 0.07091891860961914, AUC: 0.5562685


Test set: Avg. loss: 0.009477341651916504, AUC: 0.606248


Test s

In [92]:
# SIGMOID 2 CLASS undersampled

momentum=0
learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SigmoidLogisticRegression(2, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_undersampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["undersampled", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)



Test set: Avg. loss: 0.015905908584594727, AUC: 0.437009


Test set: Avg. loss: 0.053795656204223635, AUC: 0.6535575


Test set: Avg. loss: 0.1587593765258789, AUC: 0.501994


Test set: Avg. loss: 0.01805019474029541, AUC: 0.6331034999999999


Test set: Avg. loss: 0.1484091567993164, AUC: 0.5029924999999998


Test set: Avg. loss: 0.009051459789276123, AUC: 0.8229664999999999


Test set: Avg. loss: 0.02131798267364502, AUC: 0.591936


Test set: Avg. loss: 0.12902787017822265, AUC: 0.504987


Test set: Avg. loss: 0.13112792205810547, AUC: 0.507501


Test set: Avg. loss: 0.01427377700805664, AUC: 0.693999


Test set: Avg. loss: 0.06267182540893555, AUC: 0.6296015


Test set: Avg. loss: 0.00982688331604004, AUC: 0.8241054999999999


Test set: Avg. loss: 0.014709303379058838, AUC: 0.45297750000000003


Test set: Avg. loss: 0.10771421813964843, AUC: 0.5169520000000001


Test set: Avg. loss: 0.12480251693725586, AUC: 0.5094899999999999


Test set: Avg. loss: 0.009759426593780518, AUC: 0.8117


Test set: Avg. loss: 0.017284958839416505, AUC: 0.4734685


Test set: Avg. loss: 0.014687091827392577, AUC: 0.47818700000000003


Test set: Avg. loss: 0.014056768894195557, AUC: 0.5093544999999999


Test set: Avg. loss: 0.01316795301437378, AUC: 0.5372545


Test set: Avg. loss: 0.020566884994506835, AUC: 0.42508200000000007


Test set: Avg. loss: 0.014414824962615968, AUC: 0.4999595


Test set: Avg. loss: 0.012933335304260253, AUC: 0.5075405


Test set: Avg. loss: 0.011820468425750732, AUC: 0.5165455


Test set: Avg. loss: 0.011909185409545898, AUC: 0.5478265


Test set: Avg. loss: 0.01024878215789795, AUC: 0.5472199999999999


Test set: Avg. loss: 0.03545813369750977, AUC: 0.5858304999999999


Test set: Avg. loss: 0.015742668628692627, AUC: 0.510902


Test set: Avg. loss: 0.013860924243927002, AUC: 0.5114894999999999


Test set: Avg. loss: 0.013291391372680664, AUC: 0.5273589999999999


Test set: Avg. loss: 0.012884458541870118, AUC: 0.5423775


Test set: Avg. loss: 0.011252122402191


Test set: Avg. loss: 0.013757198810577393, AUC: 0.5223409999999999


Test set: Avg. loss: 0.012491308689117431, AUC: 0.5308965000000001


Test set: Avg. loss: 0.01164536714553833, AUC: 0.541986


Test set: Avg. loss: 0.05268109703063965, AUC: 0.6191185


Test set: Avg. loss: 0.05134394073486328, AUC: 0.626167


Test set: Avg. loss: 0.0499963436126709, AUC: 0.633591


Test set: Avg. loss: 0.04874342918395996, AUC: 0.640828


Test set: Avg. loss: 0.0474197940826416, AUC: 0.644409


Test set: Avg. loss: 0.04612111663818359, AUC: 0.650694


Test set: Avg. loss: 0.03865525436401367, AUC: 0.480105


Test set: Avg. loss: 0.03443008232116699, AUC: 0.47356050000000005


Test set: Avg. loss: 0.030458396911621093, AUC: 0.47061650000000005


Test set: Avg. loss: 0.026622346878051758, AUC: 0.4623749999999999


Test set: Avg. loss: 0.023026430130004882, AUC: 0.46787850000000003


Test set: Avg. loss: 0.01974360656738281, AUC: 0.4644125


Test set: Avg. loss: 0.025465276718139647, AUC: 0.37267249999

In [93]:
# SIGMOID 2 CLASS over+undersampled

momentum=0
learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SigmoidLogisticRegression(2, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_sampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["both sampled", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)



Test set: Avg. loss: 0.05337999534606934, AUC: 0.49600249999999996


Test set: Avg. loss: 0.008310622453689576, AUC: 0.8229409999999999


Test set: Avg. loss: 0.022032556533813478, AUC: 0.8314815


Test set: Avg. loss: 0.020795438766479494, AUC: 0.8312719999999999


Test set: Avg. loss: 0.023313258171081545, AUC: 0.816831


Test set: Avg. loss: 0.02472116184234619, AUC: 0.815073


Test set: Avg. loss: 0.03646420097351074, AUC: 0.470191


Test set: Avg. loss: 0.01589971446990967, AUC: 0.836288


Test set: Avg. loss: 0.025143569946289062, AUC: 0.8164925000000001


Test set: Avg. loss: 0.02244034481048584, AUC: 0.8232850000000002


Test set: Avg. loss: 0.02390873432159424, AUC: 0.8161570000000001


Test set: Avg. loss: 0.026851189613342284, AUC: 0.810004


Test set: Avg. loss: 0.034110681533813476, AUC: 0.4865600000000001


Test set: Avg. loss: 0.014527852058410645, AUC: 0.847634


Test set: Avg. loss: 0.018952420234680176, AUC: 0.8347775


Test set: Avg. loss: 0.02224026870727539, AUC: 


Test set: Avg. loss: 0.0060977694988250735, AUC: 0.713497


Test set: Avg. loss: 0.015269526958465576, AUC: 0.450575


Test set: Avg. loss: 0.005683955192565918, AUC: 0.6775589999999999


Test set: Avg. loss: 0.005292999029159546, AUC: 0.700404


Test set: Avg. loss: 0.0053687052726745605, AUC: 0.709642


Test set: Avg. loss: 0.005569222450256347, AUC: 0.7146595


Test set: Avg. loss: 0.005684868335723877, AUC: 0.717832


Test set: Avg. loss: 0.051963306427001955, AUC: 0.4924895


Test set: Avg. loss: 0.005880682468414306, AUC: 0.6820695000000001


Test set: Avg. loss: 0.006101168394088745, AUC: 0.702693


Test set: Avg. loss: 0.006057300567626953, AUC: 0.709321


Test set: Avg. loss: 0.006103636503219605, AUC: 0.716093


Test set: Avg. loss: 0.005981874704360962, AUC: 0.7195864999999999


Test set: Avg. loss: 0.03933146286010742, AUC: 0.46998100000000004


Test set: Avg. loss: 0.0061132292747497555, AUC: 0.666534


Test set: Avg. loss: 0.006323894739151001, AUC: 0.6790095


Test set:


Test set: Avg. loss: 0.010636056423187256, AUC: 0.589032


Test set: Avg. loss: 0.00998073148727417, AUC: 0.5914725


Test set: Avg. loss: 0.04502298355102539, AUC: 0.6186195000000001


Test set: Avg. loss: 0.013403358459472657, AUC: 0.48017699999999996


Test set: Avg. loss: 0.0122496657371521, AUC: 0.491198


Test set: Avg. loss: 0.011295950889587403, AUC: 0.50719


Test set: Avg. loss: 0.010495823383331299, AUC: 0.5189485


Test set: Avg. loss: 0.009846956253051757, AUC: 0.5313245


Test set: Avg. loss: 0.0540279655456543, AUC: 0.49598099999999995


Test set: Avg. loss: 0.011802628517150878, AUC: 0.4863555


Test set: Avg. loss: 0.010868781566619874, AUC: 0.5042105


Test set: Avg. loss: 0.010142717838287353, AUC: 0.5194075


Test set: Avg. loss: 0.009516834735870361, AUC: 0.533965


Test set: Avg. loss: 0.008957656383514404, AUC: 0.54678


Test set: Avg. loss: 0.044719474792480465, AUC: 0.49898899999999996


Test set: Avg. loss: 0.009033169269561768, AUC: 0.614405


Test set: Avg.

In [22]:
# SIGMOID 2 CLASS weighted 

momentum=0
learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SigmoidLogisticRegression(2, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, network, optimizer, verbose=False, class_weights=weight)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["weighted", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)



Test set: Avg. loss: 0.02157968044281006, AUC: 0.6867034999999999


Test set: Avg. loss: 0.08839923477172852, AUC: 0.698482


Test set: Avg. loss: 0.05532261276245117, AUC: 0.771594


Test set: Avg. loss: 0.09743415069580078, AUC: 0.7073455


Test set: Avg. loss: 0.11138107681274415, AUC: 0.688415


Test set: Avg. loss: 0.13640005111694337, AUC: 0.6599929999999999


Test set: Avg. loss: 0.023658716201782226, AUC: 0.398378


Test set: Avg. loss: 0.052448091506958006, AUC: 0.7732169999999999


Test set: Avg. loss: 0.09756592178344727, AUC: 0.7000415


Test set: Avg. loss: 0.1643876495361328, AUC: 0.626525


Test set: Avg. loss: 0.07234828948974609, AUC: 0.7532825000000001


Test set: Avg. loss: 0.1506313934326172, AUC: 0.6473849999999999


Test set: Avg. loss: 0.04466411209106445, AUC: 0.5848144999999999


Test set: Avg. loss: 0.04493079566955566, AUC: 0.790555


Test set: Avg. loss: 0.050890846252441405, AUC: 0.7878215000000001


Test set: Avg. loss: 0.09636771774291993, AUC: 0.7050115


Test set: Avg. loss: 0.00561512041091919, AUC: 0.736377


Test set: Avg. loss: 0.006349770069122315, AUC: 0.775451


Test set: Avg. loss: 0.007801422119140625, AUC: 0.7887299999999999


Test set: Avg. loss: 0.01001564073562622, AUC: 0.7935670000000001


Test set: Avg. loss: 0.006772331953048706, AUC: 0.7783950000000001


Test set: Avg. loss: 0.02219521713256836, AUC: 0.616943


Test set: Avg. loss: 0.00567103910446167, AUC: 0.7970124999999999


Test set: Avg. loss: 0.004923522472381592, AUC: 0.787687


Test set: Avg. loss: 0.009285329818725586, AUC: 0.8153110000000001


Test set: Avg. loss: 0.008109431505203248, AUC: 0.8174030000000001


Test set: Avg. loss: 0.009222801208496095, AUC: 0.8184280000000002


Test set: Avg. loss: 0.024299232482910157, AUC: 0.6556175


Test set: Avg. loss: 0.006862485885620117, AUC: 0.7896795


Test set: Avg. loss: 0.006230347871780396, AUC: 0.7827719999999999


Test set: Avg. loss: 0.008897309780120849, AUC: 0.8061459999999999


Test set: Avg. loss: 0.009


Test set: Avg. loss: 0.009174903392791749, AUC: 0.4983055


Test set: Avg. loss: 0.008649335861206055, AUC: 0.5161884999999999


Test set: Avg. loss: 0.03852338981628418, AUC: 0.5131654999999999


Test set: Avg. loss: 0.008823648929595947, AUC: 0.7267680000000001


Test set: Avg. loss: 0.008030216932296753, AUC: 0.7369275


Test set: Avg. loss: 0.007270618200302124, AUC: 0.7439725


Test set: Avg. loss: 0.006689181804656982, AUC: 0.7518289999999999


Test set: Avg. loss: 0.0061951904296875, AUC: 0.7552835


Test set: Avg. loss: 0.06580369186401368, AUC: 0.5360860000000001


Test set: Avg. loss: 0.011504874706268311, AUC: 0.40957849999999996


Test set: Avg. loss: 0.010178947925567627, AUC: 0.44301


Test set: Avg. loss: 0.009091023921966552, AUC: 0.476949


Test set: Avg. loss: 0.008313036441802979, AUC: 0.5113445000000001


Test set: Avg. loss: 0.007778212547302246, AUC: 0.534639


Test set: Avg. loss: 0.034681669235229494, AUC: 0.4495929999999999


Test set: Avg. loss: 0.01199896097

In [23]:
NUM_CLASSES_REDUCED = 3
nums = (0, 3, 1)
ratio = (200, 20, 1)


reduced_train_CIFAR10 = class_sampling.Reduce(train_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True)
reduced_test_CIFAR10 = class_sampling.Reduce(test_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True)

ratio_train_CIFAR10 = class_sampling.Ratio(train_CIFAR10, NUM_CLASSES_REDUCED, ratio, nums=nums)
targets = ratio_train_CIFAR10.labels 
class_count = np.unique(targets, return_counts=True)[1]

weight = 1. / class_count
samples_weight = weight[targets]
samples_weight = torch.from_numpy(samples_weight)
oversampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(max(class_count) * NUM_CLASSES_REDUCED), replacement=True)
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)
undersampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * NUM_CLASSES_REDUCED), replacement=False)

weight *= max(class_count)


train_loader_reduced = DataLoader(reduced_train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_ratio = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True) 

train_loader_oversampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=oversampler)

train_loader_undersampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler)

train_loader_sampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=sampler)

test_loader_reduced = DataLoader(reduced_test_CIFAR10, batch_size=batch_size_test, shuffle=True)

In [ ]:
# Softmax 3 class normal

learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SoftmaxLogisticRegression(NUM_CLASSES_REDUCED, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_reduced, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["normal", 3, nums, (1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)


In [ ]:
# Softmax 3 class ratio

learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SoftmaxLogisticRegression(NUM_CLASSES_REDUCED, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_ratio, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["ratio", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)


In [98]:
# Softmax 3 class oversampled

learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SoftmaxLogisticRegression(NUM_CLASSES_REDUCED, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_oversampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["oversampled", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)



Test set: Avg. loss: 0.02759136454264323, AUC: 0.45233975000000004


Test set: Avg. loss: 0.03174774042765299, AUC: 0.71556


Test set: Avg. loss: 0.024263559341430666, AUC: 0.6862747499999999


Test set: Avg. loss: 0.13270845031738282, AUC: 0.6056192499999999


Test set: Avg. loss: 0.053983177185058594, AUC: 0.6308360000000001


Test set: Avg. loss: 0.05420574061075847, AUC: 0.63994625


Test set: Avg. loss: 0.0633147481282552, AUC: 0.49774975


Test set: Avg. loss: 0.0627286376953125, AUC: 0.6289874999999999


Test set: Avg. loss: 0.026620399475097655, AUC: 0.65993375


Test set: Avg. loss: 0.07573316446940104, AUC: 0.60570375


Test set: Avg. loss: 0.025286399205525716, AUC: 0.7284740000000001


Test set: Avg. loss: 0.02852927335103353, AUC: 0.7332429999999999


Test set: Avg. loss: 0.016128000259399415, AUC: 0.47398900000000005


Test set: Avg. loss: 0.05959883499145508, AUC: 0.685323


Test set: Avg. loss: 0.05680835723876953, AUC: 0.499127


Test set: Avg. loss: 0.05567755889892


Test set: Avg. loss: 0.007086555163065593, AUC: 0.6939725


Test set: Avg. loss: 0.006652998765309652, AUC: 0.68656425


Test set: Avg. loss: 0.008103957017262776, AUC: 0.68106


Test set: Avg. loss: 0.07789510854085287, AUC: 0.5253734999999999


Test set: Avg. loss: 0.011227512995402018, AUC: 0.69161175


Test set: Avg. loss: 0.008605889956156413, AUC: 0.701222


Test set: Avg. loss: 0.007929009278615316, AUC: 0.7012729999999999


Test set: Avg. loss: 0.006564739227294922, AUC: 0.6873497499999999


Test set: Avg. loss: 0.007130799293518067, AUC: 0.6951059999999999


Test set: Avg. loss: 0.16779090372721353, AUC: 0.420122


Test set: Avg. loss: 0.010558457692464192, AUC: 0.7041875


Test set: Avg. loss: 0.010410885175069173, AUC: 0.69615


Test set: Avg. loss: 0.006845006465911865, AUC: 0.7092339999999999


Test set: Avg. loss: 0.008380580107371012, AUC: 0.7001535


Test set: Avg. loss: 0.006592520236968994, AUC: 0.7044765000000001


Test set: Avg. loss: 0.0804031244913737, AUC: 0.451


Test set: Avg. loss: 0.01855265235900879, AUC: 0.627667


Test set: Avg. loss: 0.017725901285807293, AUC: 0.6326095


Test set: Avg. loss: 0.016895203272501626, AUC: 0.63418825


Test set: Avg. loss: 0.018617775599161782, AUC: 0.540304


Test set: Avg. loss: 0.02647641944885254, AUC: 0.53209525


Test set: Avg. loss: 0.024486117045084636, AUC: 0.540969


Test set: Avg. loss: 0.02307672119140625, AUC: 0.547566


Test set: Avg. loss: 0.02115683937072754, AUC: 0.5650835


Test set: Avg. loss: 0.019348943710327148, AUC: 0.57118125


Test set: Avg. loss: 0.15332138061523437, AUC: 0.506514


Test set: Avg. loss: 0.023836283365885415, AUC: 0.63452


Test set: Avg. loss: 0.021744002024332684, AUC: 0.627308


Test set: Avg. loss: 0.020054114659627278, AUC: 0.628565


Test set: Avg. loss: 0.019174705505371092, AUC: 0.628949


Test set: Avg. loss: 0.018202950795491536, AUC: 0.62999825


Test set: Avg. loss: 0.04917230733235677, AUC: 0.33498750000000005


Test set: Avg. loss: 0.025942994435628256

In [99]:
# Softmax 3 class undersampled

learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SoftmaxLogisticRegression(NUM_CLASSES_REDUCED, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_undersampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["undersampled", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)



Test set: Avg. loss: 0.06791185506184896, AUC: 0.4566004999999999


Test set: Avg. loss: 0.08671995798746744, AUC: 0.5282500000000001


Test set: Avg. loss: 0.09225210825602213, AUC: 0.566


Test set: Avg. loss: 0.0833435287475586, AUC: 0.706586


Test set: Avg. loss: 0.12462593332926432, AUC: 0.40312625


Test set: Avg. loss: 0.07695217132568359, AUC: 0.5251574999999999


Test set: Avg. loss: 0.18642843627929687, AUC: 0.49625


Test set: Avg. loss: 0.021841992060343426, AUC: 0.54775


Test set: Avg. loss: 0.03414060211181641, AUC: 0.5727819999999999


Test set: Avg. loss: 0.1417373352050781, AUC: 0.53453375


Test set: Avg. loss: 0.06535907745361329, AUC: 0.62525


Test set: Avg. loss: 0.3198079325358073, AUC: 0.4785


Test set: Avg. loss: 0.04631866073608398, AUC: 0.42072350000000003


Test set: Avg. loss: 0.13835987854003906, AUC: 0.5195000000000001


Test set: Avg. loss: 0.051361016591389974, AUC: 0.6435


Test set: Avg. loss: 0.07425342814127604, AUC: 0.6607350000000001


Test se


Test set: Avg. loss: 0.04025861612955729, AUC: 0.5220560000000001


Test set: Avg. loss: 0.020912960688273113, AUC: 0.45255475


Test set: Avg. loss: 0.0220375607808431, AUC: 0.47045675


Test set: Avg. loss: 0.019324415842692056, AUC: 0.5046815


Test set: Avg. loss: 0.014438618024190266, AUC: 0.49843249999999995


Test set: Avg. loss: 0.018100303014119468, AUC: 0.48330799999999996


Test set: Avg. loss: 0.022196496963500977, AUC: 0.49147175


Test set: Avg. loss: 0.0213782590230306, AUC: 0.47660975


Test set: Avg. loss: 0.017011202494303386, AUC: 0.47069049999999996


Test set: Avg. loss: 0.017968064626057944, AUC: 0.5514395000000001


Test set: Avg. loss: 0.015594100634256999, AUC: 0.5063630000000001


Test set: Avg. loss: 0.01593225129445394, AUC: 0.54639675


Test set: Avg. loss: 0.0748257293701172, AUC: 0.45289375


Test set: Avg. loss: 0.02488068135579427, AUC: 0.550902


Test set: Avg. loss: 0.018912328084309895, AUC: 0.5579725


Test set: Avg. loss: 0.0176573060353597, AUC: 


Test set: Avg. loss: 0.03586234410603841, AUC: 0.530854


Test set: Avg. loss: 0.03428972816467285, AUC: 0.52935225


Test set: Avg. loss: 0.03286473210652669, AUC: 0.52563


Test set: Avg. loss: 0.0316736806233724, AUC: 0.5248975


Test set: Avg. loss: 0.03038536580403646, AUC: 0.523923


Test set: Avg. loss: 0.018682617823282878, AUC: 0.49140350000000005


Test set: Avg. loss: 0.01834485181172689, AUC: 0.48672449999999995


Test set: Avg. loss: 0.018244314829508463, AUC: 0.4827665


Test set: Avg. loss: 0.01841034444173177, AUC: 0.4929812499999999


Test set: Avg. loss: 0.018288359959920248, AUC: 0.49100124999999994


Test set: Avg. loss: 0.018316246032714843, AUC: 0.49401399999999995


Test set: Avg. loss: 0.054578914642333984, AUC: 0.49975


Test set: Avg. loss: 0.05042789204915365, AUC: 0.49925


Test set: Avg. loss: 0.04630350621541341, AUC: 0.4985


Test set: Avg. loss: 0.042328463236490886, AUC: 0.49525


Test set: Avg. loss: 0.03776787439982096, AUC: 0.49125


Test set: Avg. 

In [100]:
# Softmax 3 class both sampled

learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SoftmaxLogisticRegression(NUM_CLASSES_REDUCED, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_sampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["both sampled", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)



Test set: Avg. loss: 0.016055294354756673, AUC: 0.47430125


Test set: Avg. loss: 0.03611184946695963, AUC: 0.7559492499999999


Test set: Avg. loss: 0.03562041346232096, AUC: 0.6264779999999999


Test set: Avg. loss: 0.06254090118408204, AUC: 0.62928425


Test set: Avg. loss: 0.1142723871866862, AUC: 0.55781875


Test set: Avg. loss: 0.04170296859741211, AUC: 0.72629925


Test set: Avg. loss: 0.038351935068766274, AUC: 0.41669600000000007


Test set: Avg. loss: 0.021730786641438803, AUC: 0.7741105


Test set: Avg. loss: 0.021552952448527018, AUC: 0.7625515


Test set: Avg. loss: 0.09568314615885416, AUC: 0.607448


Test set: Avg. loss: 0.022815556208292644, AUC: 0.75852225


Test set: Avg. loss: 0.02285769526163737, AUC: 0.6950555


Test set: Avg. loss: 0.023275182088216145, AUC: 0.6044660000000001


Test set: Avg. loss: 0.14629796346028645, AUC: 0.6123725


Test set: Avg. loss: 0.03382949829101563, AUC: 0.7257669999999999


Test set: Avg. loss: 0.033460545857747395, AUC: 0.719191





Test set: Avg. loss: 0.010699673334757487, AUC: 0.68692625


Test set: Avg. loss: 0.13140752156575522, AUC: 0.40731150000000005


Test set: Avg. loss: 0.009092360814412434, AUC: 0.6532545000000001


Test set: Avg. loss: 0.00915279992421468, AUC: 0.6635584999999999


Test set: Avg. loss: 0.008236936410268148, AUC: 0.68385875


Test set: Avg. loss: 0.008985475540161133, AUC: 0.68551075


Test set: Avg. loss: 0.00788552967707316, AUC: 0.6936002499999998


Test set: Avg. loss: 0.055584776560465496, AUC: 0.5012015


Test set: Avg. loss: 0.013723857243855794, AUC: 0.6519984999999999


Test set: Avg. loss: 0.010038552284240722, AUC: 0.69695825


Test set: Avg. loss: 0.008345432122548421, AUC: 0.70370825


Test set: Avg. loss: 0.007355899175008138, AUC: 0.695063


Test set: Avg. loss: 0.01339964230855306, AUC: 0.6635892500000001


Test set: Avg. loss: 0.043987809499104816, AUC: 0.5163030000000001


Test set: Avg. loss: 0.012440644900004069, AUC: 0.6470849999999999


Test set: Avg. loss: 0.009


Test set: Avg. loss: 0.02067400868733724, AUC: 0.495604


Test set: Avg. loss: 0.01999156379699707, AUC: 0.50437225


Test set: Avg. loss: 0.019627861658732097, AUC: 0.5110064999999999


Test set: Avg. loss: 0.01923505210876465, AUC: 0.5279547499999999


Test set: Avg. loss: 0.1341986287434896, AUC: 0.50725


Test set: Avg. loss: 0.021371795654296876, AUC: 0.5519390000000001


Test set: Avg. loss: 0.018644033432006837, AUC: 0.5479595


Test set: Avg. loss: 0.01800613530476888, AUC: 0.550837


Test set: Avg. loss: 0.017945244471232098, AUC: 0.55038875


Test set: Avg. loss: 0.01756190045674642, AUC: 0.5450485


Test set: Avg. loss: 0.0460149294535319, AUC: 0.50478975


Test set: Avg. loss: 0.031996718088785805, AUC: 0.46061275


Test set: Avg. loss: 0.030549344380696616, AUC: 0.4556745


Test set: Avg. loss: 0.02836615753173828, AUC: 0.46326725


Test set: Avg. loss: 0.02741913604736328, AUC: 0.4685975


Test set: Avg. loss: 0.02567314910888672, AUC: 0.47316024999999995


Test set: Avg

In [24]:
# Softmax 3 class weighted

learning_rates = [1e-5, 1e-6, 5e-7, 1e-7, 1e-8]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.SoftmaxLogisticRegression(NUM_CLASSES_REDUCED, shape=32*32*3)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_ratio, network, optimizer, verbose=False, class_weights=weight)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["weighted", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3],
            auc_mean[i][4], auc_variance[i][4],
            auc_mean[i][5], auc_variance[i][5]]
    rows.append(row)



Test set: Avg. loss: 0.04363488133748372, AUC: 0.4146269999999999


Test set: Avg. loss: 0.04962215296427409, AUC: 0.62180475


Test set: Avg. loss: 0.17878415934244793, AUC: 0.543741


Test set: Avg. loss: 0.03869622294108073, AUC: 0.6913607500000001


Test set: Avg. loss: 0.15376093037923177, AUC: 0.5556925


Test set: Avg. loss: 0.2706865743001302, AUC: 0.5105124999999999


Test set: Avg. loss: 0.09862606811523437, AUC: 0.38977124999999996


Test set: Avg. loss: 0.2495005391438802, AUC: 0.67860725


Test set: Avg. loss: 0.12525575002034506, AUC: 0.58943425


Test set: Avg. loss: 0.03223107210795085, AUC: 0.675779


Test set: Avg. loss: 0.06034561920166016, AUC: 0.5864699999999999


Test set: Avg. loss: 0.09735928090413412, AUC: 0.6044039999999999


Test set: Avg. loss: 0.09450116729736328, AUC: 0.48917175


Test set: Avg. loss: 0.21149256388346355, AUC: 0.52174275


Test set: Avg. loss: 0.03852157719930013, AUC: 0.68021375


Test set: Avg. loss: 0.08718929290771485, AUC: 0.62881575


Test set: Avg. loss: 0.012132413546244303, AUC: 0.6738875


Test set: Avg. loss: 0.10782556406656901, AUC: 0.46748075


Test set: Avg. loss: 0.01704342969258626, AUC: 0.66308925


Test set: Avg. loss: 0.02381509717305501, AUC: 0.63820125


Test set: Avg. loss: 0.011747226079305013, AUC: 0.6678249999999999


Test set: Avg. loss: 0.00783754825592041, AUC: 0.6960740000000001


Test set: Avg. loss: 0.008658348083496093, AUC: 0.7064284999999999


Test set: Avg. loss: 0.05203556950887044, AUC: 0.39201675


Test set: Avg. loss: 0.012291367212931316, AUC: 0.673396


Test set: Avg. loss: 0.017334779739379882, AUC: 0.67508725


Test set: Avg. loss: 0.010636355717976888, AUC: 0.6902425


Test set: Avg. loss: 0.011524249712626139, AUC: 0.6736275


Test set: Avg. loss: 0.010668145497639975, AUC: 0.684875


Test set: Avg. loss: 0.10393102518717448, AUC: 0.5525


Test set: Avg. loss: 0.013746212959289551, AUC: 0.6797715


Test set: Avg. loss: 0.009276555061340333, AUC: 0.706631


Test set: Avg. loss


Test set: Avg. loss: 0.027441251754760743, AUC: 0.495814


Test set: Avg. loss: 0.02583952267964681, AUC: 0.50692925


Test set: Avg. loss: 0.02429039510091146, AUC: 0.52350125


Test set: Avg. loss: 0.0811489003499349, AUC: 0.55550675


Test set: Avg. loss: 0.027510016759236654, AUC: 0.5137315


Test set: Avg. loss: 0.024729629516601563, AUC: 0.5159265


Test set: Avg. loss: 0.02257259941101074, AUC: 0.52208825


Test set: Avg. loss: 0.020779212951660155, AUC: 0.537371


Test set: Avg. loss: 0.01924815495808919, AUC: 0.543128


Test set: Avg. loss: 0.047958841959635415, AUC: 0.5


Test set: Avg. loss: 0.018643898010253908, AUC: 0.5093172499999999


Test set: Avg. loss: 0.017556528091430663, AUC: 0.5206435


Test set: Avg. loss: 0.016849175135294596, AUC: 0.53612525


Test set: Avg. loss: 0.01629000155131022, AUC: 0.55140875


Test set: Avg. loss: 0.0157201239267985, AUC: 0.5684462499999999


Test set: Avg. loss: 0.07679866536458334, AUC: 0.446039


Test set: Avg. loss: 0.019376900990

In [27]:
df = pd.DataFrame(rows, columns = col_names) 
df.head(10)

,name,num_classes,classes_used,ratio,learning_rate,mean_0,variance_0,mean_10,variance_10,mean_20,variance_20,mean_30,variance_30,mean_40,variance_40,mean_50,variance_50
0,weighted,2,"(0, 1)","(100, 1)",1.000000e-05,0.548379,0.009424,0.705224,0.006322,0.719227,0.003053,0.692638,0.002961,0.683244,0.002862,0.694694,0.001923
1,weighted,2,"(0, 1)","(100, 1)",1.000000e-06,0.534905,0.008606,0.799263,0.004559,0.810738,0.000798,0.823757,0.000192,0.824270,0.000030,0.821001,0.000095
2,weighted,2,"(0, 1)","(100, 1)",5.000000e-07,0.527536,0.008331,0.760206,0.002216,0.779845,0.000442,0.796304,0.000275,0.791114,0.000428,0.800013,0.000264
3,weighted,2,"(0, 1)","(100, 1)",1.000000e-07,0.511716,0.005766,0.687535,0.000949,0.724008,0.000250,0.727533,0.000350,0.741630,0.000171,0.748629,0.000196
4,weighted,2,"(0, 1)","(100, 1)",1.000000e-08,0.514619,0.003869,0.538144,0.009743,0.559358,0.008715,0.577911,0.007555,0.595342,0.006466,0.608298,0.005704
5,weighted,3,"(0, 3, 1)","(200, 20, 1)",1.000000e-05,0.471751,0.001913,0.651523,0.006141,0.631073,0.005076,0.685806,0.002068,0.636715,0.005314,0.638884,0.004918
6,weighted,3,"(0, 3, 1)","(200, 20, 1)",1.000000e-06,0.477657,0.002019,0.655639,0.001957,0.676633,0.001646,0.692445,0.000774,0.632403,0.004050,0.689182,0.000715
7,weighted,3,"(0, 3, 1)","(200, 20, 1)",5.000000e-07,0.483873,0.002099,0.668588,0.000139,0.675548,0.000274,0.674923,0.000722,0.686763,0.000070,0.682985,0.000321
8,weighted,3,"(0, 3, 1)","(200, 20, 1)",1.000000e-07,0.486377,0.002996,0.623684,0.001199,0.649002,0.000381,0.663992,0.000137,0.669602,0.000240,0.674974,0.000128
9,weighted,3,"(0, 3, 1)","(200, 20, 1)",1.000000e-08,0.490079,0.003195,0.538886,0.006294,0.549378,0.005560,0.559340,0.004759,0.570045,0.004029,0.579592,0.003359


In [26]:
df.to_csv('results/auc_analysis_weighted.csv', index=False)